1. Wygenerowanie jednego rozwiązania ze ścieżką
2. Próba znalezienia pośredniego połączenia między dwoma połączeniami w rozwiazaniu
3. Jeśli to rozwiązanie polepsza wynik, to akceptujemy

In [2]:
from ai_data_eng.searching.graph import *
from ai_data_eng.searching.utils import *
from ai_data_eng.searching.searchning import * 
from ai_data_eng.searching.a_star_time_opt import a_star_time_opt_light
from ai_data_eng.searching.a_star_changes_opt import a_star_changes_opt_light
from ai_data_eng.searching.heuristics import WeightedAverageTimeHeuristic, ChangeHeuristic
from pathlib import Path

DATA_DIR = Path('../data')

In [2]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])
g = Graph(connection_graph, add_constant_change_time)

C:\Users\julia\AppData\Local\Temp\ipykernel_15156\1218945369.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [3]:
from typing import Set
from queue import PriorityQueue

CRITERION = OptimizationType.TIME

def judge_t_solution(solution, visiting_stops: Set[str]):
    matched_stops = 0 
    conn_time = diff(solution[-1]['arrival_sec'], solution[-1]['departure_sec'])
    
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return conn_time / (matched_stops + 0.01)    

def judge_p_solution(solution, visiting_stops: Set[str]):
    matched_stops = 0 
    line_changes = int(np.sum([1 for (c1, c2) in zip(solution[:-1], solution[1:]) if is_conn_change(c1, c2)]))
    
    for conn in solution:
        if conn['start_stop'] in visiting_stops:
            visiting_stops.remove(conn['start_stop'])
            matched_stops += 1
    return line_changes / (matched_stops + 0.01)  
    
def naive_solution(criterion: OptimizationType, start_stop: str, visiting_stops: List[str], leave_hour: str):
    a_star = get_a_star(criterion)
    prev_stop = start_stop
    solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star(start_stop=prev_stop, goal_stop=stop, leave_hour=leave_hour)
        solution += sub_sol
        prev_stop = stop 
        # add change time 
        leave_hour = sec_to_time(sub_sol[-1]['arrival_sec'])
    solution += a_star(start_stop=prev_stop, goal_stop=start_stop, leave_hour=leave_hour)[1]
    return solution

def get_a_star(criterion: OptimizationType):
    if criterion == OptimizationType.TIME:
        return partial(a_star_time_opt_light, heuristic=WeightedAverageTimeHeuristic(), change_time=0)
    elif criterion == OptimizationType.CHANGES:
        return partial(a_star_changes_opt_light, heuristic=ChangeHeuristic(), change_time=0)


In [4]:
sol1 = naive_solution(OptimizationType.TIME, 'PL. GRUNWALDZKI', ['most Grunwaldzki', 'Poczta Główna', 'Renoma'], '08:00:00')

C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'con

In [5]:
print_path(sol1)

PL. GRUNWALDZKI [08:00:00] --- 146 ---> most Grunwaldzki [08:02:00]
most Grunwaldzki [08:02:00] --- D ---> Poczta Główna [08:04:00]
Poczta Główna [08:04:00] --- D ---> GALERIA DOMINIKAŃSKA [08:06:00]
GALERIA DOMINIKAŃSKA [08:06:00] --- D ---> Renoma [08:09:00]
Renoma [08:09:00] --- A ---> Arkady (Capitol) [08:11:00]
Arkady (Capitol) [08:13:00] --- D ---> GALERIA DOMINIKAŃSKA [08:18:00]
GALERIA DOMINIKAŃSKA [08:18:00] --- 13 ---> Urząd Wojewódzki (Impart) [08:21:00]
Urząd Wojewódzki (Impart) [08:21:00] --- 149 ---> most Grunwaldzki [08:22:00]
most Grunwaldzki [08:22:00] --- D ---> PL. GRUNWALDZKI [08:24:00]


In [6]:
judge_t_solution(sol1, ['most Grunwaldzki', 'Poczta Główna', 'Renoma'])

39.8671096345515

In [9]:
sol2 = naive_solution(OptimizationType.CHANGES, 'PL. GRUNWALDZKI', ['most Grunwaldzki', 'Poczta Główna', 'Renoma'], '08:00:00')

C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'con

In [10]:
print_path(sol2)

PL. GRUNWALDZKI [08:00:00] --- 146 ---> most Grunwaldzki [08:02:00]
most Grunwaldzki [08:02:00] --- D ---> Poczta Główna [08:04:00]
Poczta Główna [08:04:00] --- D ---> GALERIA DOMINIKAŃSKA [08:06:00]
GALERIA DOMINIKAŃSKA [08:06:00] --- D ---> Renoma [08:09:00]
Renoma [08:10:00] --- 149 ---> Dworzec Główny (Dworcowa) [08:15:00]
Dworzec Główny (Dworcowa) [08:15:00] --- 149 ---> skwer Krasińskiego [08:17:00]
skwer Krasińskiego [08:17:00] --- 149 ---> Krasińskiego [08:18:00]
Krasińskiego [08:18:00] --- 149 ---> Urząd Wojewódzki (Impart) [08:21:00]
Urząd Wojewódzki (Impart) [08:21:00] --- 149 ---> most Grunwaldzki [08:22:00]
most Grunwaldzki [08:22:00] --- 149 ---> PL. GRUNWALDZKI [08:24:00]


In [12]:
vis_stops = ['Paprotna', 'Czajkowskiego', 'Ogród Botaniczny', 'Krucza', 'Renoma', 'Kępińska']
start_stop = 'PL. GRUNWALDZKI'
leave_hour = '08:00:00'

In [13]:
sol3 = naive_solution(OptimizationType.TIME, start_stop, vis_stops, leave_hour)

C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',
C:\Users\julia\VSCode\AI-and-Data-Engineering\ai_data_eng\searching\searchning.py:105: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'con

In [14]:
judge_t_solution(sol3, vis_stops)

29.950083194675543

In [15]:
print_path(sol3)

PL. GRUNWALDZKI [08:00:00] --- 19 ---> Piastowska [08:02:00]
Piastowska [08:02:00] --- 19 ---> Górnickiego [08:04:00]
Górnickiego [08:04:00] --- 19 ---> Ogród Botaniczny [08:06:00]
Ogród Botaniczny [08:06:00] --- 128 ---> Wyszyńskiego [08:08:00]
Wyszyńskiego [08:08:00] --- 128 ---> Damrota [08:09:00]
Damrota [08:09:00] --- 128 ---> KROMERA [08:12:00]
KROMERA [08:13:00] --- 116 ---> Berenta [08:14:00]
Berenta [08:14:00] --- 116 ---> Kasprowicza [08:15:00]
Kasprowicza [08:15:00] --- 130 ---> Syrokomli [08:16:00]
Syrokomli [08:16:00] --- 130 ---> Pola [08:17:00]
Pola [08:17:00] --- 130 ---> Broniewskiego [08:18:00]
Broniewskiego [08:18:00] --- 118 ---> Bałtycka [08:20:00]
Bałtycka [08:20:00] --- 118 ---> Bezpieczna [08:21:00]
Bezpieczna [08:21:00] --- 602 ---> Paprotna [08:22:00]
Paprotna [08:25:00] --- 111 ---> Obornicka (Wołowska) [08:26:00]
Obornicka (Wołowska) [08:26:00] --- 111 ---> Bezpieczna [08:27:00]
Bezpieczna [08:27:00] --- K ---> Bałtycka [08:29:00]
Bałtycka [08:29:00] --- K -

In [17]:
g.conn_at_index(918837)

line                                            253
departure_time                             23:10:00
arrival_time                               23:11:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111981
start_stop_lon                            17.064034
end_stop_lat                              51.109283
end_stop_lon                              17.066162
departure_sec                                 83400
arrival_sec                                   83460
Name: 918837, dtype: object

In [99]:
from ai_data_eng.searching.initialization import initialize_queue

dep_time = time_to_normalized_sec(leave_hour)

# print_info = a_star_print_info(sec_to_time)
cost_so_far = {}
# if commuting A -> B, then this will be came_from_conn[B] = A so we can recreate the path
came_from_conn = {}
stop_conn = {}

prev_conn_idx = 278511
prev_stop = g.stop_as_tuple(g.rename_stop(g.conn_at_index(prev_conn_idx)))

start_stop = 'most Grunwaldzki'
goal_stop = 'Urząd Wojewódzki (Impart)'
dep_time = time_to_normalized_sec('08:36:00')
closest_set = {start_stop}
cost_func = get_cost_func(g, OptimizationType.CHANGES)

frontier = initialize_queue(g, cost_func, cost_so_far, came_from_conn, stop_conn, start_stop, dep_time)
prev_conn = g.conn_at_index(prev_conn_idx)
prev_stop = g.stop_as_tuple(g.rename_stop(prev_conn))
for i in came_from_conn:
    g.conn_graph.loc[i, 'line'] = 'MISSING'
k = stop_conn[prev_stop]
g.conn_graph.drop(k, inplace=True)
stop_conn[prev_stop] = prev_conn_idx
came_from_conn[prev_conn_idx] = None

In [100]:
cost_so_far

{('most Grunwaldzki', 51.11013606, 17.05509252): 0,
 ('most Grunwaldzki', 51.11013327, 17.0555734): 0,
 ('most Grunwaldzki', 51.11021216, 17.05538488): 0,
 ('most Grunwaldzki', 51.11013812, 17.05549333): 0}

In [101]:
came_from_conn

{-1: None, -2: None, -3: None, -4: None, 278511: None}

In [106]:
stop_conn

{('most Grunwaldzki', 51.11013606, 17.05509252): -1,
 ('most Grunwaldzki', 51.11013327, 17.0555734): -2,
 ('most Grunwaldzki', 51.11021216, 17.05538488): 278511,
 ('most Grunwaldzki', 51.11013812, 17.05549333): -4}

In [107]:
g.conn_graph.loc[-2]

line                       MISSING
departure_time                 NaN
arrival_time                   NaN
start_stop        most Grunwaldzki
end_stop          most Grunwaldzki
start_stop_lat           51.110133
start_stop_lon           17.055573
end_stop_lat             51.110133
end_stop_lon             17.055573
departure_sec                30960
arrival_sec                  30960
Name: -2, dtype: object

In [108]:
g.stop_as_tuple(g.rename_stop(g.conn_at_index(918837)))

('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162)

In [109]:
prev_conn

line                            16
departure_time            08:35:00
arrival_time              08:36:00
start_stop         PL. GRUNWALDZKI
end_stop          most Grunwaldzki
start_stop_lat           51.111444
start_stop_lon            17.05996
end_stop_lat             51.110212
end_stop_lon             17.055385
departure_sec                30900
arrival_sec                  30960
Name: 278511, dtype: object

278512, 97887, 233196


In [ ]:
g.conn_graph.loc[[240344, 97887, 278512]]

In [80]:
g.conn_graph.loc[(g.conn_graph['start_stop'] == start_stop) & (g.conn_graph['end_stop'] == goal_stop)].sort_values(by=['departure_sec'])

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
277379,16,04:57:00,04:59:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,17820,17940
107188,4,05:01:00,05:03:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18060,18180
271778,16,05:12:00,05:14:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18720,18840
107549,4,05:16:00,05:18:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,18960,19080
100605,4,05:17:00,05:19:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,19020,19140
...,...,...,...,...,...,...,...,...,...,...,...
219138,12,23:19:00,23:20:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,83940,84000
320498,19,23:24:00,23:25:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84240,84300
231995,13,23:27:00,23:28:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84420,84480
273793,16,23:32:00,23:33:00,most Grunwaldzki,Urząd Wojewódzki (Impart),51.110212,17.055385,51.108637,17.049553,84720,84780


In [105]:
for next_conn in g.get_lines_from(prev_conn['arrival_sec'], prev_stop, '', closest_set).itertuples():
    # cost of commuting start --> current and current --> next
    next_stop = (next_conn.end_stop, next_conn.end_stop_lat, next_conn.end_stop_lon)
    print(next_conn.start_stop, next_conn.end_stop, next_conn.line, cost_func(next_conn=next_conn, prev_conn=prev_conn))

most Grunwaldzki PL. GRUNWALDZKI 16 1
most Grunwaldzki Urząd Wojewódzki (Impart) 13 1
most Grunwaldzki PL. GRUNWALDZKI 12 1
most Grunwaldzki Urząd Wojewódzki (Impart) 16 0
most Grunwaldzki PL. GRUNWALDZKI D 1
most Grunwaldzki Urząd Wojewódzki (Impart) 4 1
most Grunwaldzki Poczta Główna 145 1
most Grunwaldzki PL. GRUNWALDZKI 13 1
most Grunwaldzki Poczta Główna D 1
most Grunwaldzki PL. GRUNWALDZKI 145 1
most Grunwaldzki Poczta Główna 149 1
most Grunwaldzki Urząd Wojewódzki (Impart) 12 1
most Grunwaldzki PL. GRUNWALDZKI 4 1
most Grunwaldzki Poczta Główna 146 1
most Grunwaldzki PL. GRUNWALDZKI 146 1
most Grunwaldzki PL. GRUNWALDZKI 149 1
most Grunwaldzki Urząd Wojewódzki (Impart) 19 1
most Grunwaldzki PL. GRUNWALDZKI 19 1
most Grunwaldzki Poczta Główna 241 1
most Grunwaldzki Urząd Wojewódzki (Muzeum Narodowe) 240 1
most Grunwaldzki PL. GRUNWALDZKI 241 1


In [65]:
closest_set

{'Kliniki - Politechnika Wrocławska'}

In [67]:
g.get_lines_from(prev_conn['arrival_sec'], (prev_conn['end_stop']), prev_conn['line'], closest_set)

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec


In [ ]:
def is_neighbour(dep_time, new_solution):
    return assert_connection_path(dep_time, new_solution)

def naive_search(solution, leave_hour, start_stop, visiting_stops: List[str]):
    prev_stop = start_stop
    new_solution = []
    for stop in visiting_stops:
        _, sub_sol = a_star_time_opt(prev_stop, stop, leave_hour, WeightedAverageTimeHeuristic())
        new_solution = new_solution + sub_sol
        prev_stop = 

def replace_connection(solution, dep_time, stop_from, stop_to):
    
    is_neigh = False
    while not is_neigh:
        graph, travel_solution = a_star_time_opt(stop_from, stop_to, dep_time, WeightedAverageTimeHeuristic())